In [10]:
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime

from corp_bond_modules import get_corporate_bond_holdings
from corp_bond_modules import weighted_quantile

VUTA

In [11]:
url = "https://www.vanguard.co.uk/professional/product/etf/bond/9598/usd-treasury-bond-ucits-etf-usd-accumulating"
path = "./VUTA"

get_corporate_bond_holdings(url,os.path.abspath(path))

Initializing Chrome webdriver...
Navigating to https://www.vanguard.co.uk/professional/product/etf/bond/9598/usd-treasury-bond-ucits-etf-usd-accumulating...
Extracting holdings data...
File downloaded


In [12]:
path = "./VUTA"
path += "/*"

target_cols = ['Ticker', 'Holding', 'Market_Val_Percent', 'Market_Val_USD', 
        'Face_Val_USD', 'Coupon_Percent', 'Maturity_Date']

expected_cols = ['Ticker','Holding name','% of market value',
                 'Market value','Face amount','Coupon/Yield','Maturity date']

cols = dict(zip(expected_cols,target_cols))

dateparse = lambda x: datetime.strptime(x, '%d %b %Y')

df = pd.read_excel(max(glob.glob(path), key=os.path.getctime),
    skiprows=6)[:-2].rename(columns=cols)

df['Market_Val_Percent'] = df['Market_Val_Percent'].apply(lambda x: float(x.replace('%', '')))
df['Market_Val_USD'] = df['Market_Val_USD'].apply(lambda x: float(str(x).replace('US$', '').replace(',', '')))
df['Face_Val_USD']=df['Face_Val_USD'].apply(lambda x: float(str(x).replace(',', '')))
df['Coupon_Percent']=df['Coupon_Percent'].apply(lambda x: float(str(x).replace('%', '')))
df['Maturity_Date']=df['Maturity_Date'].apply(lambda x: pd.to_datetime(x,format='%d %b %Y'))
df['Maturity_Years_FromNow']=df['Maturity_Date'].apply(lambda x:(x-datetime.today()).days/365.25 if x.year > datetime.now().year else 0)
df['AnnExpectCoupon_NomUSD'] = df['Face_Val_USD']*df['Coupon_Percent']/100

year_bins = (
    np.pad(weighted_quantile(df['Maturity_Years_FromNow'],quantiles=[0.25,0.5,0.75],sample_weight=df['Market_Val_Percent']), 
           (1, 1), mode='constant', 
           constant_values=(0, int(np.ceil(df['Maturity_Years_FromNow'].max())))
           )
)

df['Year_Category']=pd.cut(df['Maturity_Years_FromNow'],bins=year_bins)

In [13]:
df

,Ticker,Holding,Market_Val_Percent,Market_Val_USD,Face_Val_USD,Coupon_Percent,Maturity_Date,Maturity_Years_FromNow,AnnExpectCoupon_NomUSD,Year_Category
0,T,United States Treasury Note/Bond,0.9964,25318471.08,25872300.0,4.00,2034-02-15,9.245722,1034892.00,"(8.562, 30.0]"
1,T,United States Treasury Note/Bond,0.9897,25149344.06,24966000.0,4.38,2034-05-15,9.489391,1093510.80,"(8.562, 30.0]"
2,T,United States Treasury Note/Bond,0.9858,25049190.31,24633500.0,4.50,2033-11-15,8.993840,1108507.50,"(8.562, 30.0]"
3,T,United States Treasury Note/Bond,0.9037,22964526.38,23644300.0,3.88,2033-08-15,8.741958,917398.84,"(8.562, 30.0]"
4,T,United States Treasury Note/Bond,0.8835,22450837.50,23205000.0,3.88,2034-08-15,9.741273,900354.00,"(8.562, 30.0]"
...,...,...,...,...,...,...,...,...,...,...
284,T,United States Treasury Note/Bond,0.0011,27672.56,26400.0,6.50,2026-11-15,1.993155,1716.00,"(0.0, 2.366]"
285,T,United States Treasury Note/Bond,0.0010,25496.09,24400.0,5.25,2029-02-15,4.246407,1281.00,"(2.366, 4.367]"
286,T,United States Treasury Note/Bond,0.0004,8875.78,8400.0,6.63,2027-02-15,2.245038,556.92,"(0.0, 2.366]"
287,T,United States Treasury Note/Bond,0.0003,6714.50,6400.0,5.50,2028-08-15,3.742642,352.00,"(2.366, 4.367]"


In [14]:
fig = px.histogram(df,x='Coupon_Percent',y='Market_Val_Percent',color='Year_Category',nbins=50,barmode='overlay')
fig.show()

In [15]:
fig = px.ecdf(df,x='Maturity_Years_FromNow',y='Market_Val_Percent',marginal='histogram')
fig.show()

In [16]:
# Mean

df2 = df[df['Coupon_Percent']>0]
sum((df2['Market_Val_Percent']/sum(df2['Market_Val_Percent']))*df2['Coupon_Percent'])

2.948627596978796

In [17]:
# Median

weighted_quantile(df['Coupon_Percent'],quantiles=[0.5],sample_weight=df['Market_Val_Percent'])

array([3.25])

Other

In [18]:
# df3 = (
#     df[['Maturity_Years_FromNow','Market_Val_Percent']].assign(
#         Market_Val_Percent=lambda x: x['Market_Val_Percent']/sum(df['Market_Val_Percent'])).sort_values('Maturity_Years_FromNow')
# )

# fig = px.line(df3,x='Maturity_Years_FromNow', y='Market_Val_CumPerc')
# fig.show()